## Question 7 
### Fit a neural network to the Default data in the ISLR2 package. Use a single hidden layer with 10 units, and regularization. Have a look at ISLR Labs 10.9.1–10.9.2, and in class, for guidance. Compare the classification performance of yourmodel with that of logistic regression.

In [1]:
install.packages("ISLR2")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [2]:
install.packages("keras")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘Rcpp’, ‘RcppTOML’, ‘here’, ‘png’, ‘config’, ‘tfautograph’, ‘reticulate’, ‘tensorflow’, ‘tfruns’, ‘zeallot’




In [3]:
install.packages("tensorflow")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [4]:
library(ISLR2)
library(tensorflow)
library(keras)

In [5]:
summary(Default)

 default    student       balance           income     
 No :9667   No :7056   Min.   :   0.0   Min.   :  772  
 Yes: 333   Yes:2944   1st Qu.: 481.7   1st Qu.:21340  
                       Median : 823.6   Median :34553  
                       Mean   : 835.4   Mean   :33517  
                       3rd Qu.:1166.3   3rd Qu.:43808  
                       Max.   :2654.3   Max.   :73554  

In [16]:
set.seed(1234)
n_row <- nrow(Default)
n_test <- trunc(n_row/5)
test_id <- sample(1:n, n_test)
y_test <- Default$default[test_id] == 'Yes'

In [17]:
linlog.reg <- glm(default~student+balance+income,family="binomial",data=Default[-test_id,])
linlog.pred <- predict(linlog.reg, data=Default[test_id,], type='response') > 0.5
linlog.accuracy = mean(linlog.pred == y_test)
cat( 'Accuracy :', linlog.accuracy)

Accuracy : 0.95175

In [18]:
x = model.matrix(default ~. -1, data=Default)

x_train <- x[-test_id,]
g_train <- Default$default[-test_id]=='Yes'

x_test <- x[test_id,]
g_test <- Default$default[test_id] == 'Yes'

model_nn <- keras_model_sequential() %>% 
  layer_dense(units=10, activation='relu', input_shape=ncol(x)) %>%
  layer_dropout(rate=0.4) %>% 
  layer_dense(units = 1, activation='sigmoid')

In [25]:
model_nn %>% compile(
  optimizer=optimizer_rmsprop(), 
  loss='binary_crossentropy', 
  metrics='accuracy')

In [27]:
history <- model_nn %>% fit(
  x = x_train, 
  y = g_train, 
  epochs=30, 
  batch_size=128)

In [28]:
nn_pred <- predict(model_nn, x_test) > 0.5
nn.accuracy <- mean(nn_pred == g_test)
cat('Accuracy :',nn.accuracy)

Accuracy : 0.9645

Neural network test set accuracy is about 96%. This is comparable to logistic regression which had 95% accuracy.